In [1]:
import requests
import json
from datetime import datetime
import statistics
import pandas as pd

In [4]:
url = "https://testgateway2.eniaccomputing.com/api/v2/Partner?ModifiedSince=2023-03-01&PageNumber=1&RowsOfPage=1000"

# Headers to be added to the request
headers = {
    "ConnectionId": "9c92a465-b8af-452b-993f-1879a534789d",
    "xapi-key": "BEL-X2-iZcsduXjgUHv"
}

# Sending the GET request with headers
response = requests.get(url, headers=headers)
data = json.loads(response.text)

In [2]:
df = pd.read_csv("data\\solar_panel_analysis_dataset_final.csv")

id_list = df["GEM phase ID"].tolist()
longitude_list = df["Longitude"].tolist()
latitude_list = df["Latitude"].tolist()

read_coordinates_dict = {}

for i in range(len(id_list)):
    read_coordinates_dict[id_list[i]] = {"longitude":longitude_list[i], "latitude": latitude_list[i]}

In [3]:
def create_chunks(lst, chunk_size):
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]

In [4]:
chunks = create_chunks(list(read_coordinates_dict.keys()), 1000)

In [6]:
for chunk in chunks[6:]:
    final_dict = {}
    progress = ""
    for ids in chunk:
        response = requests.get(
            "https://re.jrc.ec.europa.eu/api/v5_1/seriescalc?",
            params={
                "lat": read_coordinates_dict[ids]["latitude"],
                "lon": read_coordinates_dict[ids]["longitude"],
                "outputformat": "json",
                "startyear": 2005,
                "endyear": 2015}
        )
        data = json.loads(response.text)
        if "outputs" not in data.keys():
            final_dict[ids] = "missing spatial data"
        else:
            monthly_averages = {}
            yearly_averages = {}
            full_averages = {}

            for data_point in data['outputs']['hourly']:
                date_string = data_point['time']
                data_point['time'] = datetime.strptime(date_string, '%Y%m%d:%H%M')
                #date = data_point['time'].date()
                year = data_point['time'].strftime('%Y')
                if year == '2004':
                    continue
                month = data_point['time'].strftime('%m')
                radiance = data_point['G(i)']
                sun_height = data_point['H_sun']
                temperature = data_point['T2m']
                wind_speed = data_point['WS10m']
                parameter_dict = {'temperature': temperature, 'radiance': radiance, 'sun_height': sun_height, 'wind_speed':wind_speed}

                if month not in monthly_averages:
                    monthly_averages[month] = {}

                if year not in yearly_averages:
                    yearly_averages[year] = {}

                for items, parameter_values in parameter_dict.items():
                    if items in monthly_averages[month].keys():
                        monthly_averages[month][items].append(parameter_values)
                    else:
                        monthly_averages[month][items] = [parameter_values]


                    if items in yearly_averages[year].keys():
                        yearly_averages[year][items].append(parameter_values)
                    else:
                        yearly_averages[year][items] = [parameter_values]


                    if items in full_averages.keys():
                        full_averages[items].append(parameter_values)
                    else:
                        full_averages[items] = [parameter_values]


            for date_list in [yearly_averages, monthly_averages, full_averages]:
                for date, values in date_list.items():
                    if date_list == full_averages:
                        values = full_averages
                    for items, parameter_values in parameter_dict.items():
                        parameter_sum = sum(values[items])
                        parameter_count = len(values[items])
                        average_parameter = parameter_sum/parameter_count
                        median_parameter = statistics.median(values[items])
                        mode_parameter = statistics.mode(values[items])
                        stdev_parameter = statistics.stdev(values[items])
                        variance_parameter = statistics.variance(values[items])
                        max_parameter = max(values[items])
                        min_parameter = min(values[items])
                        range_parameter = max_parameter - min_parameter
                        values[items] = {"average": average_parameter, "median": median_parameter, "mode": mode_parameter, "standard_deviation": stdev_parameter, "variance": variance_parameter, "max": max_parameter, "min": min_parameter, "range": range_parameter}
                    if date_list == full_averages:
                        break
            final_dict[ids] = {"monthly": monthly_averages, "yearly":yearly_averages, "full": full_averages}
            current_progress = round((chunk.index(ids)/len(chunk)) * 100)
            if current_progress % 10 == 0 and current_progress != progress:
                print("{} - Progress of chunk {}/{}: {}%".format(datetime.now(),chunks.index(chunk) + 1, len(chunks), current_progress))
                progress = current_progress
    dict_for_json = {item:value for item, value in final_dict.items() if value != "missing spatial data"}
    with open('additional_weather_data\\solar_data_{}.json'.format(chunks.index(chunk) + 1), 'w') as json_file:
        json.dump(dict_for_json, json_file)

2023-06-07 12:52:14.279620 - Progress of chunk 7/9: 0%
2023-06-07 13:03:43.588562 - Progress of chunk 7/9: 10%
2023-06-07 13:16:24.304522 - Progress of chunk 7/9: 20%
2023-06-07 13:29:24.441334 - Progress of chunk 7/9: 30%
2023-06-07 13:42:31.051587 - Progress of chunk 7/9: 40%
2023-06-07 13:55:21.673638 - Progress of chunk 7/9: 50%
2023-06-07 14:08:06.991568 - Progress of chunk 7/9: 60%
2023-06-07 14:21:03.489763 - Progress of chunk 7/9: 70%
2023-06-07 14:34:03.908007 - Progress of chunk 7/9: 80%
2023-06-07 14:46:41.714717 - Progress of chunk 7/9: 90%
2023-06-07 14:59:02.402643 - Progress of chunk 7/9: 100%
2023-06-07 14:59:41.778403 - Progress of chunk 8/9: 0%
2023-06-07 15:11:19.922815 - Progress of chunk 8/9: 10%
2023-06-07 15:23:23.417152 - Progress of chunk 8/9: 20%
2023-06-07 15:35:36.988771 - Progress of chunk 8/9: 30%
2023-06-07 15:47:35.155143 - Progress of chunk 8/9: 40%
2023-06-07 15:59:43.342790 - Progress of chunk 8/9: 50%
2023-06-07 16:11:55.141299 - Progress of chunk 8/